In [ ]:
!pip install -q dm-haiku spektral optax neptune flax torch torch-geometric jraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 39.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB

In [ ]:
pip install -q -U jax jaxlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 MB 9.3 MB/s eta 0:00:00


In [ ]:
!pip install -U jax[cuda11_cudnn82] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
INFO: pip is looking at multiple versions of jax[cuda11-cudnn82] to determine which version is compatible with other requirements. This could take a while.
  Using cached jax-0.4.10-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 MB 7.6 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.8-py3-none-any.whl size=1439678 sha256=65560fab0bf57579bc302d7d3a5

In [ ]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
import neptune
import statistics
import itertools
import pickle
import networkx as nx
from typing import Any, Callable, Dict, List, Optional, Tuple
import jraph
import haiku as hk
import functools
import time

import numpy as np
from typing import Sequence


# Models

## Tokenizer

In [ ]:
from collections import defaultdict

class Tokenizer:
    def __init__(self):
        self.token2idx = defaultdict(lambda: len(self.token2idx))
        self.idx2token = {}

    def fit_on_texts(self, texts):
        for text in texts:
            self.tokenize(text)
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}

    def tokenize(self, texts):
        all_tokens = set()
        for text in texts:
            tokens = text.split()
            all_tokens.update(tokens)

        # Create token to index mapping
        self.token2idx = {token: idx for idx, token in enumerate(sorted(all_tokens))}
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}

        # Tokenize texts
        tokenized_texts = [[self.token2idx[token] for token in text.split()] for text in texts]

        return tokenized_texts

    def detokenize(self, sequences):
        # Detokenize sequences
        detokenized_texts = [" ".join(self.idx2token[token] for token in sequence) for sequence in sequences]

        return detokenized_texts

# Initialize the tokenizer and fit on the texts
tokenizer = Tokenizer()

texts = [
    "PUSH1 0x0 CALLDATALOAD PUSH29 0x100000000000000000000000000000000000000000000000000000000 SWAP1 DIV PUSH4 0xffffffff AND DUP1 PUSH4 0xda51bbd9 EQ PUSH1 0x44 JUMPI",
    "JUMPDEST PUSH1 0x0 SLOAD DUP2 JUMP",
    "PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x4 CALLDATASIZE LT PUSH1 0x3f JUMPI",
]

tokenized_texts = tokenizer.tokenize(texts)

# # Tokenize the texts
# tokenized_texts = [tokenizer.tokenize(text) for text in texts]

# # Detokenize the texts
# detokenized_texts = [tokenizer.detokenize(tokens) for tokens in tokenized_texts]

# print(tokenized_texts)
# print(detokenized_texts)


## Autoencoder


In [ ]:
class TransformerBlock(hk.Module):
  def __init__(self, hidden_dim, num_heads, dropout_rate, name=None):
    super().__init__(name=name)
    self.hidden_dim = hidden_dim
    self.num_heads = num_heads
    self.dropout_rate = dropout_rate

  def __call__(self, x, mask):
    attention = hk.MultiHeadAttention(num_heads=self.num_heads, key_size=self.hidden_dim, w_init=hk.initializers.TruncatedNormal(1.0), name='attention')
    feed_forward = hk.nets.MLP([self.hidden_dim, self.hidden_dim], name='feed_forward')

    mask = mask[:, None, None, :]  # reshape the mask to [batch_size, 1, 1, sequence_length]
    attn_output = attention(query=x, key=x, value=x, mask=mask)
    attn_output = hk.Linear(self.hidden_dim, name="linear")(attn_output)  # Apply the linear transformation
    attn_output = hk.Linear(x.shape[-1], name="linear_after_attn")(attn_output)  # New Linear layer to match the dimensions of x
    x = x + hk.dropout(hk.next_rng_key(), self.dropout_rate, attn_output)
    ffn_output = feed_forward(x)
    ffn_output = hk.Linear(x.shape[-1], name="linear_after_ffn")(ffn_output)  # Linear layer to match the dimensions of x
    ffn_output = hk.dropout(hk.next_rng_key(), self.dropout_rate, ffn_output)
    x = x + ffn_output

    return x



class TransformerAutoEncoder(hk.Module):
  def __init__(self, hidden_dim, num_layers, num_heads, dropout_rate, bottleneck_dim, name=None):
    super().__init__(name=name)
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers
    self.num_heads = num_heads
    self.dropout_rate = dropout_rate
    self.bottleneck_dim = bottleneck_dim

  def positional_encoding(self, length, d_model):
    angle_rads = self.get_angles(np.arange(length)[:, np.newaxis],
                                  np.arange(d_model)[np.newaxis, :],
                                  d_model)

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return jnp.array(pos_encoding, dtype=float)

  def get_angles(self, pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

  def __call__(self, x, mask):
    positional_encoding = self.positional_encoding(x.shape[1], x.shape[2])

    x = x + positional_encoding
    x = hk.dropout(hk.next_rng_key(), self.dropout_rate, x)

    # Encoder
    hidden_dim = self.hidden_dim
    for i in range(self.num_layers//2):  # Half of the layers are used for encoding
      x = TransformerBlock(hidden_dim, self.num_heads, self.dropout_rate, name=f'encoder_layer_{i}')(x, mask)
      hidden_dim = hidden_dim // 2  # Reduce the hidden dimension by a factor of 1/2 for each layer

    # Bottleneck
    x = hk.Linear(self.bottleneck_dim, name='bottleneck')(x)

    # Transform bottleneck output back to hidden_dim before feeding to the decoder
    x = hk.Linear(hidden_dim, name='expand')(x)

    # Decoder
    for i in range(self.num_layers//2, self.num_layers):  # Half of the layers are used for decoding
      hidden_dim = hidden_dim * 2  # Increase the hidden dimension by a factor of 2 for each layer
      x = TransformerBlock(hidden_dim, self.num_heads, self.dropout_rate, name=f'decoder_layer_{i}')(x, mask)

    # Output layer to reconstruct the original input size
    output = hk.Linear(self.hidden_dim, name='output')(x)  # Here, self.hidden_dim represents the original input size

    return output

In [ ]:
import functools
def model_fn(rng, x, mask):
  return TransformerAutoEncoder(hidden_dim=256, num_layers=2, num_heads=8, dropout_rate=0.1, bottleneck_dim=128)(x, mask)


rng = jax.random.PRNGKey(0)

model_fn_partial = functools.partial(model_fn, rng)

# Transform the model function into a pair of pure functions
model = hk.transform(model_fn_partial)

# Define the loss function
def loss_fn(params, x, mask):
  x_recon = model.apply(params, x, mask)
  loss = jnp.mean((x_recon - x) ** 2)
  return loss

# Create the optimizer
optimizer = optax.adam(0.001)

# Initialize the parameters

params = model.init(rng, jnp.ones((1, 10, 256)), jnp.zeros((1, 10), dtype=bool))  # Assuming mask is a boolean tensor



In [ ]:
import haiku as hk
import jax
from jax import numpy as jnp
from jax.nn import one_hot

def encoder_forward(hidden_dim, vocab_size, x):
    x_one_hot = one_hot(x, vocab_size)
    x_flat = x_one_hot.reshape((x.shape[0], -1))
    return hk.Linear(hidden_dim)(x_flat)

def decoder_forward(hidden_dim, vocab_size, max_seq_length, z):
    decoded = hk.Linear(max_seq_length * vocab_size)(z)
    return decoded.reshape((-1, max_seq_length, vocab_size))

class Encoder(hk.Module):
    def __init__(self, hidden_dim, vocab_size, max_seq_length, name=None):
        super().__init__(name=name)
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.max_seq_length = max_seq_length

    def __call__(self, x):
        return encoder_forward(self.hidden_dim, self.vocab_size, x)

class Decoder(hk.Module):
    def __init__(self, hidden_dim, vocab_size, max_seq_length, name=None):
        super().__init__(name=name)
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.max_seq_length = max_seq_length

    def __call__(self, z):
        return decoder_forward(self.hidden_dim, self.vocab_size, self.max_seq_length, z)


In [ ]:
# Define the model
def autoencoder_fn(x, hidden_dim, vocab_size, max_seq_length):
    # Encoder
    x_one_hot = one_hot(x, vocab_size)
    x_flat = x_one_hot.reshape((x.shape[0], -1))
    z = hk.Linear(hidden_dim)(x_flat)
    # Decoder
    decoded = hk.Linear(max_seq_length * vocab_size)(z)
    x_reconstructed = decoded.reshape((-1, max_seq_length, vocab_size))
    return x_reconstructed

def cross_entropy(log_probs, targets):
    # Get the log-probabilities of the targets
    log_probs_target = jnp.take_along_axis(log_probs, targets[..., None], axis=-1)
    # Compute the mean negative log-probability
    return -jnp.mean(log_probs_target)

def reconstruction_loss(x, params, rng):
    logits = apply_fn(params, rng, x)
    log_probs = jax.nn.log_softmax(logits, axis=-1)
    # Compute the loss for each example in the batch
    losses = vmap(cross_entropy)(log_probs, x)
    # Compute the mean loss over the batch
    return jnp.mean(losses)

In [ ]:
from jax import grad, jit, vmap
from functools import partial

# Get the maximum sequence length from the tokenized data
max_seq_length = max(len(seq) for seq in tokenized_texts)

# Pad the sequences to max_seq_length
tokenized_texts_padded = [seq + [0] * (max_seq_length - len(seq)) for seq in tokenized_texts]

# Convert to numpy arrays
tokenized_texts_padded = jnp.array(tokenized_texts_padded)
print(tokenized_texts_padded)

# Transform the model into a function that can be called with parameters and inputs
# This gives us a pair of pure functions (init and apply)
hidden_dim = 128
vocab_size = 10000
max_seq_length = 50
autoencoder_fn_partial = partial(autoencoder_fn, hidden_dim=hidden_dim, vocab_size=vocab_size, max_seq_length=max_seq_length)
init_fn, apply_fn = hk.transform(autoencoder_fn_partial)

# Initialize parameters
rng = jax.random.PRNGKey(0)  # Replace with your own key, if necessary
x = jnp.ones((1, max_seq_length), dtype=jnp.int32)  # Replace with your actual data
params = init_fn(rng, x)

# Run inference
rng_inference = jax.random.PRNGKey(1)  # Different RNG for inference
x_reconstructed = apply_fn(params, rng_inference, x)

# Compute loss
loss = reconstruction_loss(x, params, rng_inference)
print('Reconstruction loss:', loss)

[[21  0 10 22  1 25 12 23  8  9 13 23  7 15 21  5 18]
 [17 21  0 24 14 16  0  0  0  0  0  0  0  0  0  0  0]
 [21  6 21  4 20 21  3 11 19 21  2 18  0  0  0  0  0]]
Reconstruction loss: 9.208459


In [ ]:
import jax
from jax import numpy as jnp
from flax import linen as nn
from jax.nn import one_hot

class Encoder(nn.Module):
    hidden_dim: int
    vocab_size: int
    max_seq_length: int

    def setup(self):
        self.dense = nn.Dense(self.hidden_dim)

    def __call__(self, x):
        x_one_hot = one_hot(x, self.vocab_size)
        x_flat = x_one_hot.reshape((x.shape[0], -1))
        return self.dense(x_flat)

class Decoder(nn.Module):
    hidden_dim: int
    vocab_size: int
    max_seq_length: int

    def setup(self):
        self.dense = nn.Dense(self.max_seq_length * self.vocab_size)

    def __call__(self, z):
        decoded = self.dense(z)
        return decoded.reshape((-1, self.max_seq_length, self.vocab_size))

In [ ]:
# Get the maximum sequence length from the tokenized data
max_seq_length = max(len(seq) for seq in tokenized_texts)

# Pad the sequences to max_seq_length
tokenized_texts_padded = [seq + [0] * (max_seq_length - len(seq)) for seq in tokenized_texts]

# Convert to numpy arrays
tokenized_texts_padded = jnp.array(tokenized_texts_padded)
print(tokenized_texts_padded)


# Initialize the models
encoder = Encoder(hidden_dim=128, vocab_size=len(tokenizer.token2idx), max_seq_length=max_seq_length)
decoder = Decoder(hidden_dim=128, vocab_size=len(tokenizer.token2idx), max_seq_length=max_seq_length)

# Initialize parameters
encoder_params = encoder.init(jax.random.PRNGKey(0), tokenized_texts_padded)
decoder_params = decoder.init(jax.random.PRNGKey(1), jnp.ones((len(tokenized_texts_padded), 128)))  # assuming the output of the encoder is of size (batch_size, 128)

# Compile the models
encode = jax.jit(encoder.apply)
decode = jax.jit(decoder.apply)

# Encode and decode
encoded = encode(encoder_params, tokenized_texts_padded)
decoded = decode(decoder_params, encoded)

# Get the most probable token IDs
decoded_ids = jnp.argmax(decoded, axis=-1)

x = [seq.tolist() for seq in decoded_ids]
print(x)

# Detokenize
detokenized_texts = tokenizer.detokenize(x)
texts = [
    "PUSH1 0x0 CALLDATALOAD PUSH29 0x100000000000000000000000000000000000000000000000000000000 SWAP1 DIV PUSH4 0xffffffff AND DUP1 PUSH4 0xda51bbd9 EQ PUSH1 0x44 JUMPI",
    "JUMPDEST PUSH1 0x0 SLOAD DUP2 JUMP",
    "PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x4 CALLDATASIZE LT PUSH1 0x3f JUMPI",
]

print("\n".join(detokenized_texts))


[[21  0 10 22  1 25 12 23  8  9 13 23  7 15 21  5 18]
 [17 21  0 24 14 16  0  0  0  0  0  0  0  0  0  0  0]
 [21  6 21  4 20 21  3 11 19 21  2 18  0  0  0  0  0]]
[[13, 4, 13, 25, 5, 20, 17, 10, 1, 14, 3, 19, 1, 13, 13, 15, 12], [14, 4, 0, 23, 15, 2, 11, 5, 21, 22, 6, 10, 11, 21, 16, 7, 19], [5, 3, 20, 20, 25, 19, 14, 12, 11, 22, 4, 18, 17, 21, 1, 7, 11]]
DUP1 0x40 DUP1 SWAP1 0x44 MSTORE JUMPDEST CALLDATALOAD 0x100000000000000000000000000000000000000000000000000000000 DUP2 0x4 LT 0x100000000000000000000000000000000000000000000000000000000 DUP1 DUP1 EQ DIV
DUP2 0x40 0x0 PUSH4 EQ 0x3f CALLDATASIZE 0x44 PUSH1 PUSH29 0x80 CALLDATALOAD CALLDATASIZE PUSH1 JUMP 0xda51bbd9 LT
0x44 0x4 MSTORE MSTORE SWAP1 LT DUP2 DIV CALLDATASIZE PUSH29 0x40 JUMPI JUMPDEST PUSH1 0x100000000000000000000000000000000000000000000000000000000 0xda51bbd9 CALLDATASIZE


In [ ]:
def cross_entropy_loss(logits, targets):
    """
    Compute cross-entropy loss.
    Args:
        logits: [batch, length, num_classes] float array.
        targets: [batch, length] integer array.
    Returns:
        scalar float loss.
    """
    one_hot_targets = jax.nn.one_hot(targets, logits.shape[-1])
    log_probs = jax.nn.log_softmax(logits, axis=-1)
    return -jnp.sum(one_hot_targets * log_probs) / targets.size

# Compute loss
loss = cross_entropy_loss(decoded, tokenized_texts_padded)
print(loss)

3.2794256


# Data

# Training